# Extracting model outputs

***

<img src="images/wavegbr.jpg" alt="geometry" width="800" height="550"/>

*** 


> To compare output volume from the simulation with observed volume, it is necessary to convert the offshore volumes into grain volume by subtracting the pore space (here Sofia's used 54% in situ porosity) and a weighted average onshore porosity (~33%). 

<div class="alert alert-block alert-info">In this notebook, we will extract different parameters from our simulation and do some basic post-processing analyses.</div>

Some of the things that could be explore:

+ how high sediment loads from catchments erosion influenced coral growth during the transgression phase,
+ what is the relation between inland incision and sediment gravity-flows in the deepest parts of the basin floor; 
+ what are the relationships between climate, sea-level, and margin physiography and how it enabled coral reefs to thrive.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action = "ignore", category = FutureWarning)

import numpy as np
import cmocean as cmo
from matplotlib import cm

from scripts import analyseWaveReef as wrAnalyse

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

## Loading the dataset

We first load the simulation last output file (timestep=22)

In [ ]:
dataTIN = wrAnalyse.analyseWaveReef(folder='output',timestep=22)

In [ ]:
dataTIN.regridTINdataSet()

We can plot a series of different parameters from the model for the chosen timestep:

+ elevation: `dataTIN.z`
+ slope: `dataTIN.slp`
+ total erosion/deposition: `dataTIN.dz`
+ wave-induced erosion/deposition: `dataTIN.wavesed`
+ wave-induced shear stress: `dataTIN.wavestress`
+ hillslope induced erosion/deposition : `dataTIN.cumhill`
+ reef present/absent: `dataTIN.depreef`
+ aspect: `dataTIN.aspect`
+ horizontal curvature: `dataTIN.hcurv`
+ vertical curvature: `dataTIN.vcurv`

In [ ]:
dataTIN.plotdataSet(title='Elevation', data=dataTIN.z, color=cmo.cm.delta,crange=[-2000,2000])

In [ ]:
dataTIN.plotdataSet(title='Total erosion/deposition [m]', data=dataTIN.dz, color=cmo.cm.balance,crange=[-20.,20.])

In [ ]:
dataTIN.plotdataSet(title='Wave erosion/deposition [m]', data=dataTIN.wavesed, color=cmo.cm.balance,crange=[-5.,5.])

In [ ]:
dataTIN.plotdataSet(title='Hillslope erosion/deposition [m]', data=dataTIN.cumhill, color=cmo.cm.balance,
                      crange=[-2.5,2.5])

In [ ]:
dataTIN.plotdataSet(title='Wave induced stress', data=dataTIN.wavestress, color=cmo.cm.tempo,crange=[0.,2.],ctr='k')

In [ ]:
dataTIN.plotdataSet(title='Reef absent/present', data=dataTIN.depreef, color=cm.Blues,
                      crange=[0,2])

## Analysing sediment erosion/deposition and reef growth

We have access to the deposited volume over time and can perform with IPython notebook calculation of the accumulated volume in the entire region or in a specific area. This can be done in this way:


We will use the following area for this example:

***

<img src="images/geo.jpg" alt="geometry" width="800" height="550"/>

*** 


We first define the area where we will perform our analyse:

+ `erange`: [100000,250000,540000,640000]

and plot the total sedimentation change (erosion & deposition) `dataTIN.dz`.

In [ ]:
dataTIN.plotdataSet(title='Erosion/Deposition [m]', data=dataTIN.dz, color=cmo.cm.balance,  
                      crange=[-25,25], erange=[100000,250000,540000,640000],
                      depctr=(-2,10),ctr='k',size=(10,10))

Then we look if there is any reef in the region... we should have some based on the GoogleMap image above!

In [ ]:
dataTIN.plotdataSet(title='Reef position', data=dataTIN.depreef, color=cmo.cm.balance,  
                      crange=[-2,2], erange=[100000,250000,540000,640000],
                      depctr=(1),ctr='k',size=(10,10))

We can now compute a series of information regarding erosion/deposition volume for the selected region.

### Total volume of erosion and deposition

In [ ]:
dataTIN.getDepositedVolume(data=dataTIN.dz,time=11000.,erange=[100000,250000,540000,640000])
print('----')
dataTIN.getErodedVolume(data=dataTIN.dz,time=11000.,erange=[100000,250000,540000,640000])

### Total volume of erosion and deposition induced by hillslope processes

In [ ]:
dataTIN.getDepositedVolume(data=dataTIN.cumhill,time=11000.,erange=[100000,250000,540000,640000])
print('----')
dataTIN.getErodedVolume(data=dataTIN.cumhill,time=11000.,erange=[100000,250000,540000,640000])

### Total volume of erosion and deposition induced by wave processes

In [ ]:
dataTIN.getDepositedVolume(data=dataTIN.wavesed,time=11000.,erange=[100000,250000,540000,640000])
print('----')
dataTIN.getErodedVolume(data=dataTIN.wavesed,time=11000.,erange=[100000,250000,540000,640000])

### Total volume of erosion and deposition around reef systems

In [ ]:
r,c = np.where(dataTIN.depreef>0)
erodepreef = np.zeros(dataTIN.dz.shape)
erodepreef[r,c] = dataTIN.dz[r,c]
dataTIN.getDepositedVolume(data=erodepreef,time=11000.,erange=[100000,250000,540000,640000])
print('----')
dataTIN.getErodedVolume(data=erodepreef,time=11000.,erange=[100000,250000,540000,640000])

## Temporal analyse of reef growth

Up to now, we have only looked at the last time step, obviously we have access to all these information over time in our model. 

Here is an example of how to plot deposition and erosion rate through time on the reefs only. It can give some information on the timing of the reefs _turn-on_ for example in this particular area. 

This can be done in this way:

In [ ]:
time = np.linspace(-10000.0, 0.0, num=21)
time,ero,depo = wrAnalyse.temporalGrowth(folder='output',step=22,vtime=time,
                                         smooth=100,vrange=[100000,250000,540000,640000])
wrAnalyse.temporalPlot(time,ero,depo,size=(8,5))

# Additional comparisons...

***

<img src="images/obs.jpg" alt="geometry" width="1000" height="550"/>

*** 

3D view of the GBR between Cairns and Cooktown highlighting sediment transport from mountain ranges to the coast, and from the coast to the Queensland Trough. On the inner shelf dominant wave direction (SE) rework sediments to the North (black arrows). On the mid shelf, coral reef develops. Sediment transfer across the slope happens through V-shaped canyons. White lines (1–2–3) mark the location of seismic lines in b. b) Topas seismic section illustrating sediment gravity flow (SGF) and thin-bedded deposits for Canyon 8. c) 3D view of the Noggin region (canyons and slope) with draped GLORIA side-scan sonar backscatter imagery. High-reflectivity (white toned) areas correspond to SGFs and landslide deposits (b and c are adapted from Puga-Bernabeu et al.)